
Final_Bert == 765 Features Extracted

In [1]:
!pip install transformers
import transformers as ppb 
import re
from bs4 import BeautifulSoup 
import numpy as np
import torch
import pandas as pd 
import nltk
nltk.download('stopwords')
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.2 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
import os 
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedShuffleSplit

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]') 
#STOPWORDS =nltk.corpus.stopwords.words('english')

In [4]:
path = "/content/drive/MyDrive/newpro/threeclass_problem/data.csv"
df = pd.read_csv(path)
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names) 

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel,ppb.BertTokenizer, 'bert-base-cased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t), df.iloc[:,0])) 

In [ ]:
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids,train_tokens))

In [ ]:
b=np.zeros((np.shape(df)[0]))
for i in range(0,np.shape(df)[0]):
  b[i]=np.shape(train_tokens_ids[i])[0]

In [ ]:
pad=np.zeros((np.shape(df)[0],int(np.max(b))))
for i in range(0,np.shape(df)[0]):
  a=train_tokens_ids[i]
  for j in range(0,np.shape(a)[0]):
    pad[i,j]=a[j]

In [ ]:
pad1=pad[:,0:150]       #pad1=pad[:,0:300]
input_ids = torch.tensor(np.array(pad1))        
with torch.no_grad():
  last_hidden_states = model(input_ids.long()) 
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
features

array([[ 0.2515822 ,  0.06544148,  0.03632676, ...,  0.43549427,
        -0.04873884,  0.02371806],
       [ 0.30734295,  0.04622684,  0.07097933, ...,  0.33745626,
         0.15762301,  0.42273486],
       [ 0.3303049 ,  0.21474215,  0.51723254, ..., -0.03585206,
        -0.29864782,  0.32629228],
       ...,
       [ 0.17179213, -0.04870527, -0.00776932, ...,  0.08644193,
         0.2850138 ,  0.40955907],
       [ 0.28869393, -0.01770299, -0.14143702, ...,  0.10606311,
         0.26678324,  0.417692  ],
       [ 0.19270174, -0.14026037, -0.39399457, ..., -0.12173665,
         0.10634034,  0.5625396 ]], dtype=float32)

In [ ]:
len(features)

1744

In [ ]:
DF = pd.DataFrame(features)
from google.colab import files
DF.to_csv('bert_levelsrecode.csv') 
files.download('bert_levelsrecode.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
path = "/content/drive/MyDrive/newpro/threeclass_problem/bert_levelsrecode.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,0,0.251582,0.065441,0.036327,-0.683447,-0.864267,0.043688,-0.182814,-0.056845,0.180367,...,0.731047,0.057088,0.038299,0.289821,-0.283759,0.252248,0.018195,0.435494,-0.048739,0.023718
1,1,0.307343,0.046227,0.070979,-0.689943,-0.421657,-0.045189,0.146107,-0.190608,0.162453,...,0.758684,0.373472,0.056404,0.079221,-0.368649,0.135729,-0.081308,0.337456,0.157623,0.422735
2,2,0.330305,0.214742,0.517233,-0.529509,-0.374165,-0.236629,0.780357,-0.248850,0.348099,...,0.462937,0.052816,-0.340460,0.361714,-0.405706,0.020661,-0.041318,-0.035852,-0.298648,0.326292
3,3,0.266967,0.254776,0.249469,-0.518748,-0.460016,-0.099519,0.359582,0.087548,0.356397,...,0.631750,0.190572,0.242963,0.207410,-0.466586,-0.032446,0.130334,0.219458,0.072517,0.343598
4,4,0.221523,0.013097,-0.097723,-1.067212,-0.829449,0.143097,-0.051330,-0.085799,0.035921,...,0.565033,0.261153,-0.027681,0.528556,-0.489401,0.421287,-0.566908,0.184935,0.217472,0.156604


In [12]:
x.shape

(1744, 769)

In [13]:
x.drop(x.columns[[0]], axis=1, inplace=True)

In [14]:
x.shape 

(1744, 768)

In [15]:
x.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.251582,0.065441,0.036327,-0.683447,-0.864267,0.043688,-0.182814,-0.056845,0.180367,-0.370679,...,0.731047,0.057088,0.038299,0.289821,-0.283759,0.252248,0.018195,0.435494,-0.048739,0.023718
1,0.307343,0.046227,0.070979,-0.689943,-0.421657,-0.045189,0.146107,-0.190608,0.162453,-0.529358,...,0.758684,0.373472,0.056404,0.079221,-0.368649,0.135729,-0.081308,0.337456,0.157623,0.422735
2,0.330305,0.214742,0.517233,-0.529509,-0.374165,-0.236629,0.780357,-0.248850,0.348099,-0.530329,...,0.462937,0.052816,-0.340460,0.361714,-0.405706,0.020661,-0.041318,-0.035852,-0.298648,0.326292
3,0.266967,0.254776,0.249469,-0.518748,-0.460016,-0.099519,0.359582,0.087548,0.356397,-0.599168,...,0.631750,0.190572,0.242963,0.207410,-0.466586,-0.032446,0.130334,0.219458,0.072517,0.343598
4,0.221523,0.013097,-0.097723,-1.067212,-0.829449,0.143097,-0.051330,-0.085799,0.035921,0.215113,...,0.565033,0.261153,-0.027681,0.528556,-0.489401,0.421287,-0.566908,0.184935,0.217472,0.156604


In [16]:
y=df['Level ']

In [17]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 768), (524, 768))

In [18]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


Perceptron

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.4677362527969374


In [ ]:
import warnings
warnings.filterwarnings("ignore")

**Decision Tree**

In [19]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [20]:
clf_dt.fit(X_train,y_train) 

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [21]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [22]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))


In [23]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe) 

In [24]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-23-5d369e75a43b>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.59918,0.59918,0.670211,0.580022,0.44084,0.44084,0.470472,0.415861


**Random Forest**

In [25]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [26]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [27]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [28]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.6231    0.5159    0.5645       157
           2     0.5108    0.7004    0.5907       237
           3     0.5362    0.2846    0.3719       130

    accuracy                         0.5420       524
   macro avg     0.5567    0.5003    0.5090       524
weighted avg     0.5507    0.5420    0.5286       524



In [29]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-23-5d369e75a43b>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.59918,0.59918,0.670211,0.580022,0.440840,0.440840,0.470472,0.415861
1,RandomForest,1.00000,1.00000,1.000000,1.000000,0.541985,0.541985,0.550736,0.528567


**AdaBoost**

In [31]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6772    0.5928    0.6322       361
           2     0.5721    0.6962    0.6281       553
           3     0.6883    0.5196    0.5922       306

    accuracy                         0.6213      1220
   macro avg     0.6459    0.6029    0.6175      1220
weighted avg     0.6323    0.6213    0.6203      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5106    0.4586    0.4832       157
           2     0.5103    0.6287    0.5633       237
           3     0.5055    0.3538    0.4163       130

    accuracy                         0.5095       524
   macro avg     0.5088    0.4804    0.4876       524
weighted avg     0.5092    0.5095    0.5028       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.646721,0.646721,0.670006,0.631384,0.496183,0.496183,0.496357,0.476152
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.562977,0.562977,0.574905,0.554659
2,Adaboost,0.621311,0.621311,0.632336,0.620285,0.509542,0.509542,0.509197,0.502847


**Gradient Boosting Classifier**

In [33]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9723    0.9723    0.9723       361
           2     0.9597    0.9910    0.9751       553
           3     0.9965    0.9379    0.9663       306

    accuracy                         0.9721      1220
   macro avg     0.9762    0.9671    0.9712      1220
weighted avg     0.9727    0.9721    0.9721      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5870    0.5159    0.5492       157
           2     0.5191    0.6878    0.5917       237
           3     0.5972    0.3308    0.4257       130

    accuracy                         0.5477       524
   macro avg     0.5678    0.5115    0.5222       524
weighted avg     0.5588    0.5477    0.5378       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.646721,0.646721,0.670006,0.631384,0.496183,0.496183,0.496357,0.476152
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.562977,0.562977,0.574905,0.554659
2,Adaboost,0.621311,0.621311,0.632336,0.620285,0.509542,0.509542,0.509197,0.502847
3,GBM,0.972131,0.972131,0.972674,0.972067,0.547710,0.547710,0.558816,0.537757


**SVM Model**

In [34]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [35]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [36]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [37]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6860    0.8532    0.7605       361
           2     0.8213    0.6401    0.7195       553
           3     0.7353    0.8170    0.7740       306

    accuracy                         0.7475      1220
   macro avg     0.7475    0.7701    0.7513      1220
weighted avg     0.7597    0.7475    0.7453      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5256    0.7197    0.6075       157
           2     0.5160    0.4093    0.4565       237
           3     0.4959    0.4615    0.4781       130

    accuracy                         0.5153       524
   macro avg     0.5125    0.5302    0.5140       524
weighted avg     0.5139    0.5153    0.5071       524



In [38]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

<ipython-input-23-5d369e75a43b>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.599180,0.599180,0.670211,0.580022,0.440840,0.440840,0.470472,0.415861
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.541985,0.541985,0.550736,0.528567
2,SVM,0.747541,0.747541,0.759704,0.745304,0.515267,0.515267,0.513857,0.507093


**KNN**

In [40]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6438    0.6510    0.6474       361
           2     0.6204    0.7269    0.6694       553
           3     0.6860    0.4641    0.5536       306

    accuracy                         0.6385      1220
   macro avg     0.6501    0.6140    0.6235      1220
weighted avg     0.6438    0.6385    0.6339      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5429    0.6051    0.5723       157
           2     0.5130    0.5823    0.5455       237
           3     0.4750    0.2923    0.3619       130

    accuracy                         0.5172       524
   macro avg     0.5103    0.4932    0.4932       524
weighted avg     0.5125    0.5172    0.5080       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.646721,0.646721,0.670006,0.631384,0.496183,0.496183,0.496357,0.476152
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.562977,0.562977,0.574905,0.554659
2,Adaboost,0.621311,0.621311,0.632336,0.620285,0.509542,0.509542,0.509197,0.502847
3,GBM,0.972131,0.972131,0.972674,0.972067,0.547710,0.547710,0.558816,0.537757
4,SVM,0.762295,0.762295,0.768117,0.761422,0.576336,0.576336,0.577599,0.569378
5,KNN,0.638525,0.638525,0.643773,0.633861,0.517176,0.517176,0.512523,0.507958


**GNB**

In [42]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3847    0.7950    0.5185       361
           2     0.4552    0.1193    0.1891       553
           3     0.4316    0.4641    0.4472       306

    accuracy                         0.4057      1220
   macro avg     0.4238    0.4595    0.3850      1220
weighted avg     0.4284    0.4057    0.3513      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3676    0.7962    0.5030       157
           2     0.5455    0.1266    0.2055       237
           3     0.4419    0.4385    0.4402       130

    accuracy                         0.4046       524
   macro avg     0.4517    0.4537    0.3829       524
weighted avg     0.4665    0.4046    0.3528       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.646721,0.646721,0.670006,0.631384,0.496183,0.496183,0.496357,0.476152
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.562977,0.562977,0.574905,0.554659
2,Adaboost,0.621311,0.621311,0.632336,0.620285,0.509542,0.509542,0.509197,0.502847
3,GBM,0.972131,0.972131,0.972674,0.972067,0.547710,0.547710,0.558816,0.537757
4,SVM,0.762295,0.762295,0.768117,0.761422,0.576336,0.576336,0.577599,0.569378
5,KNN,0.638525,0.638525,0.643773,0.633861,0.517176,0.517176,0.512523,0.507958
6,GNB,0.405738,0.405738,0.428415,0.351328,0.404580,0.404580,0.466479,0.352848
